# Agenda

1. Functions
    - Inner functions
    - Dispatch tables
    - Bytecodes and compilation
2. Mypy + annotations
    - Type annotations / type hints
    - Mypy 
    - Modern techniques
3. Objects
    - What happens when we create an object?
    - Attributes (ICPO)
    - Methods 
    - Magic methods
    - Inheritance